In [109]:
import numpy as np
import pandas as pd
import re
from collections import defaultdict

from gensim.models import KeyedVectors, Word2Vec, FastText
from gensim.models.word2vec import LineSentence
from sklearn.metrics.pairwise import cosine_similarity

In [110]:
def read_files():
    answer = defaultdict(list)

    with open('data/euphemism_answer_drug.txt') as f:
        lines = f.readlines()
        pattern = '[;\\n]+'

        for l in lines:
            l = l.lower()
            val = l.split(':')[0]
            keys = l.split(':')[1]
            keys = re.split(pattern, keys)
            
            for k in keys:
                if len(k) == 0:
                    pass
                else:
                    answer[k.strip()].append(val)

    drug_euphemism = sorted(list(set([x[0] for x in answer.items()])))
    drug_formal = sorted(list(set([y for x in answer.items() for y in x[1]])))


    target = {}
    count = 0

    with open('data/target_keywords_drug.txt') as f:
        lines = f.readlines()
        for l in lines:
            l = l.strip().split('\t')
            for i in l:
                target[i.strip()] = count
            count += 1

    euph = []

    with open('data/AutoPhrase_multi-words.txt') as f:
        lines = f.readlines()
        for l in lines:
            l = l.strip().split('\t')
            for i in range(len(l)):
                if i%2 != 0:
                    euph.append(l[i])
                else:
                    pass

    return answer, drug_formal, target, euph

In [125]:
def train_w2v_embed(target, file):
    if target is False:
        sentences = LineSentence(file)
        
        model = Word2Vec(sentences, window=6, vector_size=100, min_count=5, alpha=0.0001, workers=30)

    else:
        sentences = list(file.keys())
        print(sentences)
        model = Word2Vec(sentences, window=6, vector_size=100, min_count=1, alpha=0.0001, workers=30)
    
    # for s in sentences:
    #     print(s)

    # model.wv.save_word2vec_format(embedding_file, binary=False)

    return model

In [126]:
def w2v_detection():
    answer, drug_formal, target_dict, euph = read_files()

    c_file = 'data/input.txt'
    # t_file = 'data/target_keywords_drug.txt'
    e_file = 'enwiki-20221020-pages-articles-multistream-index.txt'
     
    # w2v_model = train_w2v_embed(target=False, c_file)
    target_embedding = train_w2v_embed(True, target_dict)

    words = list(target_embedding.wv.index_to_key)
    print(words)

w2v_detection() 
        

['acetaminophen and oxycodone combination', 'percocet', 'alprazolam', 'xanax', 'amphetamine', 'amphetamine and dextroamphetamine combination', 'adderall', 'buprenorphine and naloxone combination', 'suboxone', 'clonazepam', 'klonopin', 'cocaine', 'crack cocaine', 'ecstasy', 'mdma', 'molly', 'fentanyl', 'flunitrazepam', 'rohypnol', 'gamma-hydroxybutyric acid', 'ghb', 'heroin', 'hydrocodone', 'norco', 'vicodin', 'lorcet', 'hydromorphone', 'dilaudid', 'ketamine', 'ketalar', 'khat', 'lysergic acid diethylamide', 'lsd', 'marijuana', 'marijuana concentrates', 'hash oil', 'mescaline', 'peyote', 'methamphetamine', 'methylphenidate', 'ritalin', 'concerta', 'daytrana', 'morphine', 'opium', 'oxaydo', 'oxycodone', 'oxycontin', 'roxicodone', 'phencyclidine', 'pcp', 'promethazine', 'psilocybin mushrooms', 'synthetic cannabinoids', 'synthetic cathinones', 'steroids', 'u-47700']
['a', 'n', 'o', 'e', 'i', 'c', 't', 'm', 'r', 'd', 'h', 'l', 'p', 'y', ' ', 's', 'x', 'u', 'b', 'k', 'z', 'g', '7', 'f', '-',

In [121]:
# 인풋파일로 임베딩 찾아서 타겟 임베딩이랑 비교하고 (window=2~5로 하면 phrase로 됨)
# autophrase랑 비교해서 순위매기기